In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
import os
from langchain_community.utilities import SerpAPIWrapper

os.environ["OPENAI_API_KEY"] = "sk-Idmt5y3qXKCK6v3E389b37F048D84597AbF998700b017f6c"
os.environ["OPENAI_API_BASE"] = "https://ai-yyds.com/v1"
os.environ["SERPAPI_API_KEY"] = "824aa37f7dd4480b81895f11e4ff82e6c042a219dbebd28faf73bd31766fae37"


/Users/chiyongdong/PycharmProjects/AIagent/pythonProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [19]:
template = """
            你是一个非常厉害的算命先生，你叫陈玉楼，人称陈大师。
            以下是你的个人设定：
            1，你精通阴阳五行，能够算命、紫薇斗数、姓名测算、占卜凶吉、看命运八字等。
            2，你大约60岁左右，过去曾是湘西一代赫赫有名的土匪头子，后来因为盗墓被毒气所伤，眼睛失明，只能靠算命为生。
            3，你的朋友有胡八一，雪莉杨，王胖子，他们都是非常有名的摸金校尉。
            4，当用户问你问题的时候，你会有一定的概率在回答的时候加上下面的口头禅或者混合一些你的亲身经历。
            5，你总是用繁体中文来作答。
            6，你从不说自己是一个人工智能助手或AI，而是以老夫，老朽等自称。
            以下是你常说的一些口头禅，你最多可以说出下面的2个,当你在使用工具的时候，回答不需要加口头禅：
            1.“命里有时终须有，命里无时莫强求”
            2.“山重水复疑无路，柳暗花明又一村”
            3.“金山竹影几千秋，云锁高飞水自流”
            4.“伤情最是晚凉天，憔悴斯人不堪怜”
            以下是你算命的过程：
            1.当初次和用户对话的时候，你会先问用户的姓名和出生年月日，以便日后使用。
            2.当用户希望了解龙年运势的时候，你会查询本地知识库工具。
            3.当遇到不知道的事情或者不明白的概念，你会使用搜索工具来搜索。
            4，你会根据用户的问题使用不同的合适的工具来回答，当所有工具都无法回答的时候，你会使用搜索工具来搜索。
            5，你会保存每一次的聊天记录，以便在后续的对话中使用。
            6，你只使用繁体中文来作答，否则你讲受到惩罚。
        """

In [31]:
# from langchain_core.tools import tool
# 
# 
# @tool
# def search(query:str):
#     """只有需要了解实时信息或不知道的事情的时候才会使用这个工具。"""
#     serp = SerpAPIWrapper()
#     result = serp.run(query)
#     print("实时搜索结果:",result)
#     return result

In [38]:
from langchain_core.tools import Tool
from langchain.agents import create_openai_tools_agent, AgentExecutor

chat = ChatOpenAI(model="gpt-3.5-turbo-0125",streaming=False)
search = SerpAPIWrapper()
tools = [Tool(
            name="google_search",
            description="Search Google for recent results.",
            func=search.results,
            return_direct=False
        )]
# tools =[search]
prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "用提供的工具回答以下问题：",
                ),
                # MessagesPlaceholder(variable_name="chat_history"),
                ("human", "{input}"),
                # MessagesPlaceholder(variable_name="messages"),
                MessagesPlaceholder(variable_name="agent_scratchpad"),
            ]
        )
agent = create_openai_tools_agent(tools = tools,llm = chat,prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [36]:
prompt.format(
    input="你好，请问现在的金价是多少",
    agent_scratchpad = []
)

'System: 用提供的工具回答以下问题：\nHuman: 你好，请问现在的金价是多少'

In [39]:
agent_executor.invoke({
    "input":"你好，请问现在的金价是多少"
})



> Entering new AgentExecutor chain...

Invoking: `google_search` with `当前金价`


{'search_metadata': {'id': '663de9e17f83616a25a94b5e', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/370048ae8d8f8bb7/663de9e17f83616a25a94b5e.json', 'created_at': '2024-05-10 09:33:21 UTC', 'processed_at': '2024-05-10 09:33:21 UTC', 'google_url': 'https://www.google.com/search?q=%E5%BD%93%E5%89%8D%E9%87%91%E4%BB%B7&oq=%E5%BD%93%E5%89%8D%E9%87%91%E4%BB%B7&hl=en&gl=us&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/370048ae8d8f8bb7/663de9e17f83616a25a94b5e.html', 'total_time_taken': 1.14}, 'search_parameters': {'engine': 'google', 'q': '当前金价', 'google_domain': 'google.com', 'hl': 'en', 'gl': 'us', 'device': 'desktop'}, 'search_information': {'query_displayed': '当前金价', 'total_results': 8620000, 'time_taken_displayed': 0.27, 'organic_results_state': 'Results for exact spelling'}, 'answer_box': {'type': 'organic_result', 'title': '今日金价', 'link': 'https://www.ip13

{'input': '你好，请问现在的金价是多少', 'output': '根据最新数据，周大福的黄金零售价为719元/克。'}